# 📊 한국투자증권 API를 활용한 ETF 종합 분석 노트북 (보강판)

## 주요 개선사항
- ✅ 미정의 변수 문제 해결
- ✅ Plotly 시각화 추가
- ✅ bt 라이브러리 백테스팅 연동
- ✅ 멀티 ETF 비교 기능
- ✅ 에러 처리 강화
- ✅ 재사용 가능한 클래스 구조


## 1️⃣ 필수 라이브러리 설치


In [1]:
# ETF 분석 및 시각화, 백테스팅 라이브러리 설치
!pip install pandas numpy plotly bt yfinance requests python-dotenv quantstats


## 2️⃣ 라이브러리 임포트 및 환경 설정


In [2]:
import os
import json
import requests
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from pathlib import Path
from dotenv import load_dotenv
from typing import Dict, List, Optional, Tuple, Any
from dataclasses import dataclass, field, asdict
import warnings
warnings.filterwarnings('ignore')

# 시각화
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots

# .env에서 KIS 설정 읽기
load_dotenv()


True

## 3️⃣ 설정 및 유틸리티 클래스


In [3]:
@dataclass
class KISConfig:
    """KIS API 설정"""
    url_base: str = ""
    app_key: str = ""
    app_secret: str = ""
    access_token: str = ""
    
    @classmethod
    def from_env(cls) -> 'KISConfig':
        return cls(
            url_base=os.getenv("KIS_URL_BASE", "https://openapi.koreainvestment.com:9443"),
            app_key=os.getenv("KIS_APP_KEY", ""),
            app_secret=os.getenv("KIS_APP_SECRET", ""),
            access_token=os.getenv("KIS_ACCESS_TOKEN", "")
        )


def to_float(value: Any, default: Optional[float] = None) -> Optional[float]:
    """문자열 숫자 → float 변환 (콤마/공백 안전 처리)"""
    if value is None:
        return default
    try:
        return float(str(value).replace(",", "").strip())
    except (ValueError, TypeError):
        return default


def to_int(value: Any, default: Optional[int] = None) -> Optional[int]:
    """문자열 숫자 → int 변환"""
    if value is None:
        return default
    try:
        return int(float(str(value).replace(",", "").strip()))
    except (ValueError, TypeError):
        return default


def safe_divide(a: Optional[float], b: Optional[float], default: Optional[float] = None) -> Optional[float]:
    """안전한 나눗셈"""
    if a is None or b is None or b == 0:
        return default
    return a / b



## 4️⃣ KIS API 클라이언트 클래스


In [4]:
class KISAPIClient:
    """한국투자증권 API 클라이언트"""
    
    # TR_ID 상수
    TR_ETF_PRICE = "FHPST02400000"          # ETF 현재가
    TR_STOCK_INFO = "CTPF1002R"             # 주식 기본 조회
    TR_DAILY_PRICE = "FHKST03010100"        # 일/주/월/년봉
    TR_COMPONENT = "FHKST121600C0"          # ETF 구성종목
    TR_ORDERBOOK = "FHKST01010200"          # 호가/예상체결
    
    def __init__(self, config: KISConfig):
        self.config = config
        self._validate_config()
    
    def _validate_config(self):
        """설정 유효성 검사"""
        if not self.config.app_key or not self.config.app_secret:
            raise ValueError("❌ APP_KEY 또는 APP_SECRET이 설정되지 않았습니다.")
    
    def _get_headers(self, tr_id: str) -> Dict[str, str]:
        """API 공통 헤더 생성"""
        return {
            "content-type": "application/json; charset=utf-8",
            "authorization": f"Bearer {self.config.access_token}",
            "appkey": self.config.app_key,
            "appsecret": self.config.app_secret,
            "tr_id": tr_id,
            "custtype": "P",
        }
    
    def _request(self, method: str, endpoint: str, tr_id: str, 
                 params: Dict = None, data: Dict = None, 
                 timeout: int = 10, retries: int = 3) -> Dict:
        """API 요청 (재시도 로직 포함)"""
        url = f"{self.config.url_base}{endpoint}"
        headers = self._get_headers(tr_id)
        
        for attempt in range(retries):
            try:
                if method.upper() == "GET":
                    response = requests.get(url, headers=headers, params=params, timeout=timeout)
                else:
                    response = requests.post(url, headers=headers, json=data, timeout=timeout)
                
                result = response.json()
                
                # 토큰 만료 체크
                if result.get("rt_cd") in ["EGW00123", "EGW00121"]:
                    print(f"⚠️ 토큰 만료 감지, 재발급 시도... (시도 {attempt + 1}/{retries})")
                    if self.refresh_token():
                        headers = self._get_headers(tr_id)
                        continue
                    else:
                        raise RuntimeError("토큰 재발급 실패")
                
                return result
                
            except requests.exceptions.Timeout:
                print(f"⚠️ 요청 타임아웃 (시도 {attempt + 1}/{retries})")
                if attempt == retries - 1:
                    raise
            except requests.exceptions.RequestException as e:
                print(f"⚠️ 요청 실패: {e} (시도 {attempt + 1}/{retries})")
                if attempt == retries - 1:
                    raise
        
        return {}
    
    def refresh_token(self) -> bool:
        """액세스 토큰 재발급"""
        url = f"{self.config.url_base}/oauth2/tokenP"
        headers = {"content-type": "application/json"}
        body = {
            "grant_type": "client_credentials",
            "appkey": self.config.app_key,
            "appsecret": self.config.app_secret
        }
        
        try:
            response = requests.post(url, headers=headers, json=body, timeout=10)
            data = response.json()
            
            if response.status_code == 200 and data.get("access_token"):
                self.config.access_token = data["access_token"]
                print(f"✅ 토큰 재발급 성공 (만료: {data.get('access_token_token_expired', 'N/A')})")
                return True
            else:
                print(f"❌ 토큰 재발급 실패: {data}")
                return False
        except Exception as e:
            print(f"❌ 토큰 재발급 중 오류: {e}")
            return False
    
    def validate_token(self) -> bool:
        """토큰 유효성 검사"""
        try:
            result = self._request(
                "GET",
                "/uapi/etfetn/v1/quotations/inquire-price",
                self.TR_ETF_PRICE,
                params={"FID_COND_MRKT_DIV_CODE": "J", "FID_INPUT_ISCD": "069500"}
            )
            return result.get("rt_cd") == "0"
        except:
            return False



## 5️⃣ ETF 데이터 모델


In [5]:
@dataclass
class ETFBasicInfo:
    """ETF 기본 정보"""
    code: str
    name: str = ""
    listing_date: Optional[datetime] = None
    days_since_listing: Optional[int] = None
    years_since_listing: Optional[float] = None
    category: str = ""
    representative_sector: str = ""
    
    
@dataclass
class ETFPrice:
    """ETF 가격 정보"""
    current_price: Optional[float] = None
    prev_close: Optional[float] = None
    open_price: Optional[float] = None
    high_price: Optional[float] = None
    low_price: Optional[float] = None
    change: Optional[float] = None
    change_pct: Optional[float] = None
    

@dataclass
class ETFNav:
    """ETF NAV 정보"""
    nav: Optional[float] = None
    nav_change: Optional[float] = None
    nav_change_pct: Optional[float] = None
    premium: Optional[float] = None  # 괴리율
    tracking_error: Optional[float] = None
    

@dataclass
class ETFLiquidity:
    """ETF 유동성 정보"""
    volume: Optional[int] = None
    prev_volume: Optional[int] = None
    turnover: Optional[float] = None  # 거래대금 (원)
    turnover_billion: Optional[float] = None  # 거래대금 (억원)
    avg_20d_volume: Optional[float] = None
    avg_20d_turnover_billion: Optional[float] = None
    spread_abs: Optional[float] = None  # 스프레드 (원)
    spread_pct: Optional[float] = None  # 스프레드 (%)


@dataclass  
class ETFAUM:
    """ETF 순자산 정보"""
    aum_total: Optional[float] = None  # 원
    aum_billion: Optional[float] = None  # 억원
    aum_trillion: Optional[float] = None  # 조원
    circulating_shares: Optional[int] = None
    

@dataclass
class ETFComponent:
    """ETF 구성종목"""
    code: str
    name: str
    weight_pct: float
    price: Optional[float] = None
    valuation: Optional[float] = None
    market_cap: Optional[float] = None
    sector: str = ""
    is_domestic: bool = True


@dataclass
class ETFRiskMetrics:
    """ETF 위험 지표"""
    volatility_daily: Optional[float] = None
    volatility_annual: Optional[float] = None
    mdd: Optional[float] = None
    sharpe_ratio: Optional[float] = None
    sortino_ratio: Optional[float] = None
    beta: Optional[float] = None
    alpha: Optional[float] = None


@dataclass
class ETFReturns:
    """ETF 수익률"""
    return_1m: Optional[float] = None
    return_3m: Optional[float] = None
    return_6m: Optional[float] = None
    return_1y: Optional[float] = None
    return_3y: Optional[float] = None
    return_5y: Optional[float] = None
    return_ytd: Optional[float] = None
    cagr: Optional[float] = None


@dataclass
class ETFAnalysis:
    """ETF 종합 분석 결과"""
    basic_info: ETFBasicInfo
    price: ETFPrice = field(default_factory=ETFPrice)
    nav: ETFNav = field(default_factory=ETFNav)
    liquidity: ETFLiquidity = field(default_factory=ETFLiquidity)
    aum: ETFAUM = field(default_factory=ETFAUM)
    risk: ETFRiskMetrics = field(default_factory=ETFRiskMetrics)
    returns: ETFReturns = field(default_factory=ETFReturns)
    components: List[ETFComponent] = field(default_factory=list)
    quality_score: Optional[float] = None
    analysis_time: str = field(default_factory=lambda: datetime.now().isoformat())
    
    def to_dict(self) -> Dict:
        """딕셔너리로 변환"""
        return asdict(self)



## 6️⃣ ETF 분석기 클래스


In [6]:
class ETFAnalyzer:
    """ETF 종합 분석기"""
    
    RISK_FREE_RATE = 0.035  # 무위험 수익률 (한국 기준)
    
    def __init__(self, client: KISAPIClient):
        self.client = client
        self._naver_etf_cache: Dict[str, str] = {}
    
    # ==================== 기본 정보 조회 ====================
    
    def get_etf_price_info(self, etf_code: str) -> Tuple[ETFPrice, ETFNav, ETFLiquidity, ETFAUM, ETFBasicInfo]:
        """
        ETF 현재가/NAV/유동성/AUM 조회
        API: /uapi/etfetn/v1/quotations/inquire-price
        """
        result = self.client._request(
            "GET",
            "/uapi/etfetn/v1/quotations/inquire-price",
            KISAPIClient.TR_ETF_PRICE,
            params={"FID_COND_MRKT_DIV_CODE": "J", "FID_INPUT_ISCD": etf_code}
        )
        
        if result.get("rt_cd") != "0":
            raise RuntimeError(f"ETF 시세 조회 실패: {result.get('msg1')}")
        
        output = result.get("output", {})
        
        # 가격 정보
        price = ETFPrice(
            current_price=to_float(output.get("stck_prpr")),
            prev_close=to_float(output.get("stck_prdy_clpr")),
            open_price=to_float(output.get("stck_oprc")),
            high_price=to_float(output.get("stck_hgpr")),
            low_price=to_float(output.get("stck_lwpr")),
            change=to_float(output.get("prdy_vrss")),
            change_pct=to_float(output.get("prdy_ctrt"))
        )
        
        # NAV 정보
        nav_value = to_float(output.get("nav"))
        premium = None
        if price.current_price and nav_value and nav_value > 0:
            premium = (price.current_price - nav_value) / nav_value * 100
        
        nav = ETFNav(
            nav=nav_value,
            nav_change=to_float(output.get("nav_prdy_vrss")),
            nav_change_pct=to_float(output.get("nav_prdy_ctrt")),
            premium=premium,
            tracking_error=to_float(output.get("trc_errt"))
        )
        
        # 유동성 정보 (일부 - 나머지는 별도 조회 필요)
        volume = to_int(output.get("acml_vol"))
        turnover = None
        if price.current_price and volume:
            turnover = price.current_price * volume
            
        liquidity = ETFLiquidity(
            volume=volume,
            prev_volume=to_int(output.get("prdy_vol")),
            turnover=turnover,
            turnover_billion=turnover / 100_000_000 if turnover else None
        )
        
        # AUM 정보
        aum_api = to_float(output.get("etf_ntas_ttam"))  # 1억원 단위
        circ_shares = to_int(output.get("etf_crcl_stcn"))
        
        aum_total = None
        if nav_value and circ_shares:
            aum_total = nav_value * circ_shares
        elif aum_api:
            aum_total = aum_api * 100_000_000
        
        aum = ETFAUM(
            aum_total=aum_total,
            aum_billion=aum_total / 100_000_000 if aum_total else None,
            aum_trillion=aum_total / 1_000_000_000_000 if aum_total else None,
            circulating_shares=circ_shares
        )
        
        # 기본 정보
        listing_date_raw = output.get("stck_lstn_date")
        listing_date = None
        days_since = None
        years_since = None
        
        if listing_date_raw and listing_date_raw != "0":
            try:
                listing_date = datetime.strptime(listing_date_raw, "%Y%m%d")
                days_since = (datetime.now() - listing_date).days
                years_since = days_since / 365.25
            except:
                pass
        
        basic_info = ETFBasicInfo(
            code=etf_code,
            name=output.get("mbcr_name", etf_code),
            listing_date=listing_date,
            days_since_listing=days_since,
            years_since_listing=round(years_since, 2) if years_since else None,
            category=output.get("etf_div_name", ""),
            representative_sector=output.get("etf_rprs_bstp_kor_isnm", "")
        )
        
        return price, nav, liquidity, aum, basic_info
    
    def get_etf_name(self, etf_code: str) -> str:
        """ETF 이름 조회 (KIS → 네이버 폴백)"""
        # KIS에서 시도
        try:
            result = self.client._request(
                "GET",
                "/uapi/domestic-stock/v1/quotations/search-stock-info",
                "CTPF1604R",
                params={"PRDT_TYPE_CD": "512", "PDNO": etf_code}
            )
            if result.get("rt_cd") == "0" and result.get("output"):
                output = result["output"]
                if isinstance(output, list) and output:
                    output = output[0]
                name = output.get("prdt_name") or output.get("hts_kor_isnm")
                if name:
                    return name
        except:
            pass
        
        # 네이버 폴백
        return self._get_etf_name_from_naver(etf_code)
    
    def _get_etf_name_from_naver(self, etf_code: str) -> str:
        """네이버에서 ETF 이름 조회"""
        if etf_code in self._naver_etf_cache:
            return self._naver_etf_cache[etf_code]
        
        try:
            url = "https://finance.naver.com/api/sise/etfItemList.nhn"
            res = requests.get(url, headers={"User-Agent": "Mozilla/5.0"}, timeout=10)
            data = res.json()
            
            for item in data.get("result", {}).get("etfItemList", []):
                code = str(item.get("itemcode", ""))
                name = item.get("itemname", code)
                self._naver_etf_cache[code] = name
                
            return self._naver_etf_cache.get(etf_code, etf_code)
        except:
            return etf_code
    
    # ==================== 구성종목 조회 ====================
    
    def get_components(self, etf_code: str) -> List[ETFComponent]:
        """
        ETF 구성종목 조회
        API: /uapi/etfetn/v1/quotations/inquire-component-stock-price
        """
        result = self.client._request(
            "GET",
            "/uapi/etfetn/v1/quotations/inquire-component-stock-price",
            KISAPIClient.TR_COMPONENT,
            params={
                "FID_COND_MRKT_DIV_CODE": "J",
                "FID_INPUT_ISCD": etf_code,
                "FID_COND_SCR_DIV_CODE": "11216"
            }
        )
        
        if result.get("rt_cd") != "0":
            print(f"⚠️ 구성종목 조회 실패: {result.get('msg1')}")
            return []
        
        components = []
        for row in result.get("output2", []):
            code = row.get("stck_shrn_iscd", "")
            weight = to_float(row.get("etf_cnfg_issu_rlim"))
            
            if code and weight:
                components.append(ETFComponent(
                    code=code,
                    name=row.get("hts_kor_isnm", code),
                    weight_pct=weight,
                    price=to_float(row.get("stck_prpr")),
                    valuation=to_float(row.get("etf_vltn_amt")),
                    market_cap=to_float(row.get("hts_avls")),
                    is_domestic=code.isdigit() and len(code) == 6
                ))
        
        # 비중 기준 정렬
        components.sort(key=lambda x: x.weight_pct, reverse=True)
        return components
    
    def get_stock_sector(self, stock_code: str) -> str:
        """개별 종목 섹터 조회"""
        try:
            result = self.client._request(
                "GET",
                "/uapi/domestic-stock/v1/quotations/search-stock-info",
                "CTPF1002R",
                params={"PRDT_TYPE_CD": "300", "PDNO": stock_code}
            )
            if result.get("rt_cd") == "0":
                return result.get("output", {}).get("std_idst_clsf_cd_name", "-")
        except:
            pass
        return "-"
    
    # ==================== 가격 히스토리 조회 ====================
    
    def get_price_history(self, etf_code: str, days: int = 365, 
                          period: str = "D") -> pd.DataFrame:
        """
        가격 히스토리 조회
        API: /uapi/domestic-stock/v1/quotations/inquire-daily-itemchartprice
        
        Parameters:
            etf_code: ETF 종목코드
            days: 조회 일수
            period: D(일봉), W(주봉), M(월봉), Y(년봉)
        """
        all_data = []
        
        # KIS API는 한 번에 최대 100개까지 반환하므로 분할 조회
        chunk_days = 100 if period == "D" else 365 * 5
        
        end_date = datetime.now()
        start_date = end_date - timedelta(days=days)
        
        current_end = end_date
        while current_end > start_date:
            current_start = max(start_date, current_end - timedelta(days=chunk_days))
            
            result = self.client._request(
                "GET",
                "/uapi/domestic-stock/v1/quotations/inquire-daily-itemchartprice",
                KISAPIClient.TR_DAILY_PRICE,
                params={
                    "FID_COND_MRKT_DIV_CODE": "J",
                    "FID_INPUT_ISCD": etf_code,
                    "FID_INPUT_DATE_1": current_start.strftime("%Y%m%d"),
                    "FID_INPUT_DATE_2": current_end.strftime("%Y%m%d"),
                    "FID_PERIOD_DIV_CODE": period,
                    "FID_ORG_ADJ_PRC": "0"
                }
            )
            
            if result.get("rt_cd") == "0":
                for row in result.get("output2", []):
                    date_str = row.get("stck_bsop_date")
                    if date_str:
                        all_data.append({
                            "date": pd.to_datetime(date_str, format="%Y%m%d"),
                            "open": to_float(row.get("stck_oprc")),
                            "high": to_float(row.get("stck_hgpr")),
                            "low": to_float(row.get("stck_lwpr")),
                            "close": to_float(row.get("stck_clpr")),
                            "volume": to_int(row.get("acml_vol")),
                            "turnover": to_float(row.get("acml_tr_pbmn"))
                        })
            
            current_end = current_start - timedelta(days=1)
            
            # 월봉/년봉은 한 번만 조회
            if period in ["M", "Y"]:
                break
        
        if not all_data:
            return pd.DataFrame()
        
        df = pd.DataFrame(all_data)
        df = df.drop_duplicates(subset=["date"]).sort_values("date")
        df = df.set_index("date")
        
        return df
    
    # ==================== 호가/스프레드 조회 ====================
    
    def get_orderbook(self, etf_code: str) -> Dict:
        """
        호가 정보 조회
        API: /uapi/domestic-stock/v1/quotations/inquire-asking-price-exp-ccn
        """
        result = self.client._request(
            "GET",
            "/uapi/domestic-stock/v1/quotations/inquire-asking-price-exp-ccn",
            KISAPIClient.TR_ORDERBOOK,
            params={"FID_COND_MRKT_DIV_CODE": "J", "FID_INPUT_ISCD": etf_code}
        )
        
        if result.get("rt_cd") != "0":
            return {}
        
        output = result.get("output1", {})
        
        ask_price = to_float(output.get("askp1"))
        bid_price = to_float(output.get("bidp1"))
        
        spread_abs = None
        spread_pct = None
        
        if ask_price and bid_price and bid_price > 0:
            mid_price = (ask_price + bid_price) / 2
            spread_abs = ask_price - bid_price
            spread_pct = spread_abs / mid_price * 100
        
        return {
            "ask_price": ask_price,
            "bid_price": bid_price,
            "spread_abs": spread_abs,
            "spread_pct": spread_pct,
            "total_ask_volume": to_int(output.get("total_askp_rsqn")),
            "total_bid_volume": to_int(output.get("total_bidp_rsqn"))
        }
    
    # ==================== 수익률/위험 지표 계산 ====================
    
    def calculate_returns(self, prices: pd.Series) -> ETFReturns:
        """기간별 수익률 계산"""
        if prices.empty or len(prices) < 2:
            return ETFReturns()
        
        returns = ETFReturns()
        
        # 각 기간별 수익률
        periods = {
            "return_1m": 21,
            "return_3m": 63,
            "return_6m": 126,
            "return_1y": 252,
            "return_3y": 756,
            "return_5y": 1260
        }
        
        for attr, days in periods.items():
            if len(prices) >= days:
                ret = (prices.iloc[-1] / prices.iloc[-days] - 1) * 100
                setattr(returns, attr, round(ret, 2))
        
        # YTD
        year_start = pd.Timestamp(f"{datetime.now().year}-01-01")
        ytd_prices = prices[prices.index >= year_start]
        if len(ytd_prices) >= 2:
            returns.return_ytd = round((ytd_prices.iloc[-1] / ytd_prices.iloc[0] - 1) * 100, 2)
        
        # CAGR
        if len(prices) >= 252:
            years = len(prices) / 252
            total_return = prices.iloc[-1] / prices.iloc[0] - 1
            returns.cagr = round(((1 + total_return) ** (1/years) - 1) * 100, 2)
        
        return returns
    
    def calculate_risk_metrics(self, prices: pd.Series, 
                               benchmark_prices: pd.Series = None) -> ETFRiskMetrics:
        """위험 지표 계산"""
        if prices.empty or len(prices) < 20:
            return ETFRiskMetrics()
        
        daily_returns = prices.pct_change().dropna()
        
        # 변동성
        vol_daily = daily_returns.std() * 100
        vol_annual = vol_daily * np.sqrt(252)
        
        # MDD (Maximum Drawdown)
        cumulative = (1 + daily_returns).cumprod()
        running_max = cumulative.cummax()
        drawdown = (cumulative - running_max) / running_max
        mdd = drawdown.min() * 100
        
        # 샤프 비율
        excess_return = daily_returns.mean() * 252 - self.RISK_FREE_RATE
        sharpe = excess_return / (vol_annual / 100) if vol_annual > 0 else None
        
        # 소르티노 비율
        downside_returns = daily_returns[daily_returns < 0]
        downside_std = downside_returns.std() * np.sqrt(252) if len(downside_returns) > 0 else 0
        sortino = excess_return / downside_std if downside_std > 0 else None
        
        risk = ETFRiskMetrics(
            volatility_daily=round(vol_daily, 2),
            volatility_annual=round(vol_annual, 2),
            mdd=round(mdd, 2),
            sharpe_ratio=round(sharpe, 3) if sharpe else None,
            sortino_ratio=round(sortino, 3) if sortino else None
        )
        
        # 베타/알파 (벤치마크 있을 경우)
        if benchmark_prices is not None and not benchmark_prices.empty:
            common_idx = prices.index.intersection(benchmark_prices.index)
            if len(common_idx) >= 20:
                etf_ret = prices.loc[common_idx].pct_change().dropna()
                bmk_ret = benchmark_prices.loc[common_idx].pct_change().dropna()
                
                # 인덱스 정렬
                common = etf_ret.index.intersection(bmk_ret.index)
                etf_ret = etf_ret.loc[common]
                bmk_ret = bmk_ret.loc[common]
                
                if len(common) >= 20:
                    cov = etf_ret.cov(bmk_ret)
                    var = bmk_ret.var()
                    risk.beta = round(cov / var, 3) if var > 0 else None
                    
                    if risk.beta:
                        etf_annual = etf_ret.mean() * 252
                        bmk_annual = bmk_ret.mean() * 252
                        expected = self.RISK_FREE_RATE + risk.beta * (bmk_annual - self.RISK_FREE_RATE)
                        risk.alpha = round((etf_annual - expected) * 100, 3)
        
        return risk
    
    def calculate_avg_turnover(self, df: pd.DataFrame, days: int = 20) -> Tuple[Optional[float], Optional[float]]:
        """평균 거래량/거래대금 계산"""
        if df.empty or len(df) < days:
            return None, None
        
        recent = df.tail(days)
        
        # 거래대금 계산 (종가 × 거래량)
        turnovers = recent["close"] * recent["volume"]
        turnovers = turnovers.dropna()
        
        if turnovers.empty:
            return None, None
        
        avg_volume = recent["volume"].mean()
        avg_turnover = turnovers.mean()
        avg_turnover_billion = avg_turnover / 100_000_000
        
        return avg_volume, avg_turnover_billion
    
    # ==================== 종합 분석 ====================
    
    def analyze(self, etf_code: str, 
                include_components: bool = True,
                include_history: bool = True,
                benchmark_code: str = "069500") -> ETFAnalysis:
        """ETF 종합 분석 실행"""
        print(f"\n{'='*50}")
        print(f"📊 ETF 분석 시작: {etf_code}")
        print(f"{'='*50}")
        
        # 1. 기본 정보 조회
        print("1️⃣ 기본 정보 조회 중...")
        price, nav, liquidity, aum, basic_info = self.get_etf_price_info(etf_code)
        
        # 이름 보완
        if not basic_info.name or basic_info.name == etf_code:
            basic_info.name = self.get_etf_name(etf_code)
        
        # 2. 호가 정보
        print("2️⃣ 호가 정보 조회 중...")
        orderbook = self.get_orderbook(etf_code)
        if orderbook:
            liquidity.spread_abs = orderbook.get("spread_abs")
            liquidity.spread_pct = orderbook.get("spread_pct")
        
        # 3. 구성종목
        components = []
        if include_components:
            print("3️⃣ 구성종목 조회 중...")
            components = self.get_components(etf_code)
            print(f"   → {len(components)}개 종목 조회됨")
        
        # 4. 가격 히스토리 및 위험/수익률 지표
        risk = ETFRiskMetrics()
        returns = ETFReturns()
        
        if include_history:
            print("4️⃣ 가격 히스토리 조회 중...")
            daily_df = self.get_price_history(etf_code, days=365*3, period="D")
            
            if not daily_df.empty:
                # 평균 거래량/거래대금
                avg_vol, avg_turnover = self.calculate_avg_turnover(daily_df, 20)
                liquidity.avg_20d_volume = avg_vol
                liquidity.avg_20d_turnover_billion = avg_turnover
                
                # 수익률
                returns = self.calculate_returns(daily_df["close"])
                
                # 위험 지표
                print("5️⃣ 위험 지표 계산 중...")
                benchmark_df = None
                if benchmark_code and benchmark_code != etf_code:
                    benchmark_df = self.get_price_history(benchmark_code, days=365*3, period="D")
                    if not benchmark_df.empty:
                        benchmark_df = benchmark_df["close"]
                
                risk = self.calculate_risk_metrics(daily_df["close"], benchmark_df)
        
        # 5. 품질 점수 계산
        quality_score = self._calculate_quality_score(liquidity, nav, aum, risk)
        
        analysis = ETFAnalysis(
            basic_info=basic_info,
            price=price,
            nav=nav,
            liquidity=liquidity,
            aum=aum,
            risk=risk,
            returns=returns,
            components=components,
            quality_score=quality_score
        )
        
        print(f"\n✅ 분석 완료: {basic_info.name} ({etf_code})")
        return analysis
    
    def _calculate_quality_score(self, liquidity: ETFLiquidity, nav: ETFNav, 
                                  aum: ETFAUM, risk: ETFRiskMetrics) -> float:
        """ETF 품질 점수 계산 (0-100)"""
        score = 0
        weights = {"liquidity": 30, "tracking": 25, "size": 25, "risk": 20}
        
        # 유동성 점수 (스프레드 + 거래대금)
        liquidity_score = 0
        if liquidity.spread_pct is not None:
            if liquidity.spread_pct <= 0.1:
                liquidity_score += 50
            elif liquidity.spread_pct <= 0.2:
                liquidity_score += 35
            elif liquidity.spread_pct <= 0.3:
                liquidity_score += 20
            else:
                liquidity_score += 5
        
        if liquidity.avg_20d_turnover_billion is not None:
            if liquidity.avg_20d_turnover_billion >= 100:
                liquidity_score += 50
            elif liquidity.avg_20d_turnover_billion >= 50:
                liquidity_score += 40
            elif liquidity.avg_20d_turnover_billion >= 10:
                liquidity_score += 25
            else:
                liquidity_score += 10
        
        score += liquidity_score * weights["liquidity"] / 100
        
        # 추적 오차 점수
        if nav.tracking_error is not None:
            if nav.tracking_error <= 0.5:
                score += weights["tracking"]
            elif nav.tracking_error <= 1.0:
                score += weights["tracking"] * 0.8
            elif nav.tracking_error <= 2.0:
                score += weights["tracking"] * 0.5
            else:
                score += weights["tracking"] * 0.2
        
        # 규모 점수
        if aum.aum_trillion is not None:
            if aum.aum_trillion >= 1:
                score += weights["size"]
            elif aum.aum_trillion >= 0.5:
                score += weights["size"] * 0.8
            elif aum.aum_trillion >= 0.1:
                score += weights["size"] * 0.5
            else:
                score += weights["size"] * 0.2
        
        # 위험 조정 수익 점수
        if risk.sharpe_ratio is not None:
            if risk.sharpe_ratio >= 1.0:
                score += weights["risk"]
            elif risk.sharpe_ratio >= 0.5:
                score += weights["risk"] * 0.7
            elif risk.sharpe_ratio >= 0:
                score += weights["risk"] * 0.4
            else:
                score += weights["risk"] * 0.1
        
        return round(score, 1)



## 7️⃣ 시각화 클래스


In [7]:
class ETFVisualizer:
    """ETF 시각화"""
    
    @staticmethod
    def plot_price_chart(df: pd.DataFrame, title: str = "가격 차트") -> go.Figure:
        """가격 캔들 차트"""
        fig = make_subplots(
            rows=2, cols=1,
            shared_xaxes=True,
            vertical_spacing=0.03,
            subplot_titles=[title, "거래량"],
            row_heights=[0.7, 0.3]
        )
        
        # 캔들스틱
        fig.add_trace(
            go.Candlestick(
                x=df.index,
                open=df["open"],
                high=df["high"],
                low=df["low"],
                close=df["close"],
                name="가격"
            ),
            row=1, col=1
        )
        
        # 거래량 바
        colors = ["red" if close < open else "green" 
                  for close, open in zip(df["close"], df["open"])]
        fig.add_trace(
            go.Bar(x=df.index, y=df["volume"], marker_color=colors, name="거래량"),
            row=2, col=1
        )
        
        fig.update_layout(
            height=600,
            showlegend=False,
            xaxis_rangeslider_visible=False
        )
        
        return fig
    
    @staticmethod
    def plot_returns_comparison(analyses: List[ETFAnalysis]) -> go.Figure:
        """수익률 비교 차트"""
        periods = ["1개월", "3개월", "6개월", "1년", "YTD"]
        attrs = ["return_1m", "return_3m", "return_6m", "return_1y", "return_ytd"]
        
        fig = go.Figure()
        
        for analysis in analyses:
            values = [getattr(analysis.returns, attr, None) for attr in attrs]
            fig.add_trace(go.Bar(
                name=f"{analysis.basic_info.name}",
                x=periods,
                y=values,
                text=[f"{v:.1f}%" if v else "-" for v in values],
                textposition="auto"
            ))
        
        fig.update_layout(
            title="기간별 수익률 비교",
            barmode="group",
            yaxis_title="수익률 (%)",
            height=400
        )
        
        return fig
    
    @staticmethod
    def plot_risk_return_scatter(analyses: List) -> go.Figure:
        """
        위험-수익 산점도 (수정됨)
        - 샤프비율 음수 처리: 절대값으로 크기, 색상으로 양/음 구분
        """
        data = []
        for a in analyses:
            if a.risk.volatility_annual and a.returns.return_1y:
                sharpe = a.risk.sharpe_ratio or 0
                data.append({
                    "name": a.basic_info.name,
                    "code": a.basic_info.code,
                    "volatility": a.risk.volatility_annual,
                    "return": a.returns.return_1y,
                    "sharpe": sharpe,
                    "sharpe_abs": abs(sharpe),  # ✅ 버블 크기용 (항상 양수)
                    "sharpe_sign": "양수 (좋음)" if sharpe >= 0 else "음수 (나쁨)"
                })
        
        if not data:
            print("⚠️ 시각화할 데이터가 없습니다.")
            return go.Figure()
        
        df = pd.DataFrame(data)
        
        # 색상으로 샤프비율 양/음 구분
        fig = go.Figure()
        
        for sign in df["sharpe_sign"].unique():
            subset = df[df["sharpe_sign"] == sign]
            color = "green" if "양수" in sign else "red"
            
            # 각 포인트별로 hovertemplate 설정
            for _, row in subset.iterrows():
                fig.add_trace(go.Scatter(
                    x=[row["volatility"]],
                    y=[row["return"]],
                    mode="markers+text",
                    name=f"{row['code']} ({sign})",
                    text=[row["code"]],
                    textposition="top center",
                    marker=dict(
                        size=row["sharpe_abs"] * 20 + 15,  # 최소 크기 15
                        color=color,
                        opacity=0.7,
                        line=dict(width=2, color="white")
                    ),
                    hovertemplate=(
                        f"<b>{row['code']}</b><br>" +
                        f"변동성: {row['volatility']:.1f}%<br>" +
                        f"수익률: {row['return']:.1f}%<br>" +
                        f"샤프: {row['sharpe']:.3f}<extra></extra>"
                    ),
                    showlegend=False
                ))
        
        fig.update_layout(
            title="위험-수익 분석 (🟢 샤프 양수, 🔴 샤프 음수, 크기=|샤프|)",
            xaxis_title="연간 변동성 (%)",
            yaxis_title="1년 수익률 (%)",
            height=500
        )
        
        # 기준선 (수익률 0%)
        fig.add_hline(y=0, line_dash="dash", line_color="gray", opacity=0.5)
        
        return fig
    @staticmethod
    def plot_component_pie(components: List[ETFComponent], top_n: int = 10) -> go.Figure:
        """구성종목 비중 파이차트"""
        top = components[:top_n]
        other_weight = sum(c.weight_pct for c in components[top_n:])
        
        labels = [c.name for c in top]
        values = [c.weight_pct for c in top]
        
        if other_weight > 0:
            labels.append("기타")
            values.append(other_weight)
        
        fig = go.Figure(data=[go.Pie(
            labels=labels,
            values=values,
            textinfo="label+percent",
            hole=0.3
        )])
        
        fig.update_layout(
            title=f"구성종목 비중 (Top {top_n})",
            height=500
        )
        
        return fig
    
    @staticmethod
    def plot_drawdown(prices: pd.Series, title: str = "Drawdown") -> go.Figure:
        """드로우다운 차트"""
        returns = prices.pct_change().dropna()
        cumulative = (1 + returns).cumprod()
        running_max = cumulative.cummax()
        drawdown = (cumulative - running_max) / running_max * 100
        
        fig = go.Figure()
        
        fig.add_trace(go.Scatter(
            x=drawdown.index,
            y=drawdown.values,
            fill="tozeroy",
            name="Drawdown",
            line=dict(color="red")
        ))
        
        fig.update_layout(
            title=title,
            yaxis_title="Drawdown (%)",
            height=300
        )
        
        return fig
    
    @staticmethod
    def create_summary_table(analysis: ETFAnalysis) -> go.Figure:
        """요약 테이블"""
        headers = ["항목", "값"]
        
        cells = [
            ["ETF명", analysis.basic_info.name],
            ["종목코드", analysis.basic_info.code],
            ["현재가", f"{analysis.price.current_price:,.0f}원" if analysis.price.current_price else "-"],
            ["NAV", f"{analysis.nav.nav:,.0f}원" if analysis.nav.nav else "-"],
            ["괴리율", f"{analysis.nav.premium:+.3f}%" if analysis.nav.premium else "-"],
            ["추적오차", f"{analysis.nav.tracking_error:.2f}%" if analysis.nav.tracking_error else "-"],
            ["AUM", f"{analysis.aum.aum_trillion:.2f}조원" if analysis.aum.aum_trillion and analysis.aum.aum_trillion >= 1 
                    else f"{analysis.aum.aum_billion:,.0f}억원" if analysis.aum.aum_billion else "-"],
            ["스프레드", f"{analysis.liquidity.spread_pct:.3f}%" if analysis.liquidity.spread_pct else "-"],
            ["20일 평균 거래대금", f"{analysis.liquidity.avg_20d_turnover_billion:,.0f}억원" 
                                  if analysis.liquidity.avg_20d_turnover_billion else "-"],
            ["연간 변동성", f"{analysis.risk.volatility_annual:.1f}%" if analysis.risk.volatility_annual else "-"],
            ["MDD", f"{analysis.risk.mdd:.1f}%" if analysis.risk.mdd else "-"],
            ["샤프비율", f"{analysis.risk.sharpe_ratio:.3f}" if analysis.risk.sharpe_ratio else "-"],
            ["1년 수익률", f"{analysis.returns.return_1y:+.1f}%" if analysis.returns.return_1y else "-"],
            ["CAGR", f"{analysis.returns.cagr:+.1f}%" if analysis.returns.cagr else "-"],
            ["품질점수", f"{analysis.quality_score:.0f}/100" if analysis.quality_score else "-"],
        ]
        
        fig = go.Figure(data=[go.Table(
            header=dict(values=headers, fill_color="paleturquoise", align="left"),
            cells=dict(values=list(zip(*cells)), fill_color="lavender", align="left")
        )])
        
        fig.update_layout(height=500, title=f"{analysis.basic_info.name} 요약")
        
        return fig



## 8️⃣ 백테스팅 유틸리티


In [8]:
class ETFBacktester:
    """ETF 백테스팅 유틸리티"""
    
    def __init__(self, analyzer: ETFAnalyzer):
        self.analyzer = analyzer
    
    def prepare_bt_data(self, etf_codes: List[str], 
                        start_date: datetime = None,
                        end_date: datetime = None) -> pd.DataFrame:
        """
        bt 라이브러리용 데이터 준비
        
        Returns:
            종목코드를 컬럼으로 하는 종가 DataFrame
        """
        if start_date is None:
            start_date = datetime.now() - timedelta(days=365*3)
        if end_date is None:
            end_date = datetime.now()
        
        all_prices = {}
        
        for code in etf_codes:
            print(f"데이터 수집 중: {code}...")
            df = self.analyzer.get_price_history(
                code, 
                days=(end_date - start_date).days,
                period="D"
            )
            if not df.empty:
                all_prices[code] = df["close"]
        
        if not all_prices:
            return pd.DataFrame()
        
        result = pd.DataFrame(all_prices)
        result = result.loc[start_date:end_date]
        result = result.dropna(how="all")
        
        return result
    
    def run_simple_backtest(self, prices: pd.DataFrame, 
                            weights: Dict[str, float] = None) -> Dict:
        """
        간단한 백테스트 실행 (동일가중 or 지정가중)
        
        Parameters:
            prices: 종가 DataFrame
            weights: {종목코드: 비중} (None이면 동일가중)
        """
        try:
            import bt
        except ImportError:
            print("⚠️ bt 라이브러리가 설치되지 않았습니다: pip install bt")
            return {}
        
        if prices.empty:
            return {}
        
        # 가중치 설정
        if weights is None:
            weights = {col: 1.0/len(prices.columns) for col in prices.columns}
        
        # 전략 생성
        strategy = bt.Strategy(
            "ETF Portfolio",
            [
                bt.algos.RunMonthly(),
                bt.algos.SelectAll(),
                bt.algos.WeighSpecified(**weights),
                bt.algos.Rebalance()
            ]
        )
        
        # 백테스트 실행
        backtest = bt.Backtest(strategy, prices)
        result = bt.run(backtest)
        
        # 결과 추출
        stats = result.stats
        
        return {
            "total_return": float(stats.loc["total_return", "ETF Portfolio"]) * 100,
            "cagr": float(stats.loc["cagr", "ETF Portfolio"]) * 100,
            "max_drawdown": float(stats.loc["max_drawdown", "ETF Portfolio"]) * 100,
            "sharpe": float(stats.loc["yearly_sharpe", "ETF Portfolio"]),
            "volatility": float(stats.loc["yearly_vol", "ETF Portfolio"]) * 100,
            "result_object": result
        }
    
    def compare_strategies(self, prices: pd.DataFrame, 
                          strategies: Dict[str, Dict[str, float]]) -> pd.DataFrame:
        """여러 전략 비교"""
        try:
            import bt
        except ImportError:
            print("⚠️ bt 라이브러리가 필요합니다")
            return pd.DataFrame()
        
        backtests = []
        
        for name, weights in strategies.items():
            strategy = bt.Strategy(
                name,
                [
                    bt.algos.RunMonthly(),
                    bt.algos.SelectAll(),
                    bt.algos.WeighSpecified(**weights),
                    bt.algos.Rebalance()
                ]
            )
            backtests.append(bt.Backtest(strategy, prices))
        
        result = bt.run(*backtests)
        
        return result.stats.T



## 9️⃣ 실행 예시


In [9]:
def main():
    """메인 실행 함수"""
    
    # 1. 설정 로드
    print("🔧 설정 로드 중...")
    config = KISConfig.from_env()
    
    if not config.app_key:
        print("❌ .env 파일에 KIS API 키가 설정되지 않았습니다.")
        print("필요한 환경변수: KIS_URL_BASE, KIS_APP_KEY, KIS_APP_SECRET, KIS_ACCESS_TOKEN")
        return
    
    # 2. 클라이언트 초기화
    client = KISAPIClient(config)
    
    # 3. 토큰 검증
    print("🔐 토큰 검증 중...")
    if not client.validate_token():
        print("토큰 재발급 시도...")
        if not client.refresh_token():
            print("❌ 토큰 발급 실패")
            return
    print("✅ 토큰 유효")
    
    # 4. 분석기 초기화
    analyzer = ETFAnalyzer(client)
    visualizer = ETFVisualizer()
    
    # 5. 분석할 ETF 목록
    etf_codes = [
        "069500",  # KODEX 200
        "252670",  # KODEX 200선물인버스2X
        "229200",  # KODEX 코스닥150
        "305720",  # KODEX 2차전지산업
    ]
    
    # 6. 개별 ETF 분석
    analyses = []
    for code in etf_codes[:2]:  # 예시로 2개만
        try:
            analysis = analyzer.analyze(code, include_components=True)
            analyses.append(analysis)
            
            # 요약 출력
            print(f"\n📊 {analysis.basic_info.name} 요약")
            print(f"   현재가: {analysis.price.current_price:,.0f}원")
            print(f"   NAV: {analysis.nav.nav:,.0f}원 (괴리율: {analysis.nav.premium:+.3f}%)" if analysis.nav.nav else "")
            print(f"   AUM: {analysis.aum.aum_trillion:.2f}조원" if analysis.aum.aum_trillion and analysis.aum.aum_trillion >= 1 
                  else f"   AUM: {analysis.aum.aum_billion:,.0f}억원" if analysis.aum.aum_billion else "")
            print(f"   스프레드: {analysis.liquidity.spread_pct:.3f}%" if analysis.liquidity.spread_pct else "")
            print(f"   20일 평균 거래대금: {analysis.liquidity.avg_20d_turnover_billion:,.0f}억원" 
                  if analysis.liquidity.avg_20d_turnover_billion else "")
            print(f"   1년 수익률: {analysis.returns.return_1y:+.1f}%" if analysis.returns.return_1y else "")
            print(f"   연간 변동성: {analysis.risk.volatility_annual:.1f}%" if analysis.risk.volatility_annual else "")
            print(f"   MDD: {analysis.risk.mdd:.1f}%" if analysis.risk.mdd else "")
            print(f"   샤프비율: {analysis.risk.sharpe_ratio:.3f}" if analysis.risk.sharpe_ratio else "")
            print(f"   품질점수: {analysis.quality_score:.0f}/100" if analysis.quality_score else "")
            
        except Exception as e:
            print(f"❌ {code} 분석 실패: {e}")
    
    # 7. 시각화 예시 (Jupyter에서 실행 시)
    if analyses:
        print("\n📈 시각화 생성 중...")
        
        # 수익률 비교
        fig_returns = visualizer.plot_returns_comparison(analyses)
        fig_returns.show()
        
        # 위험-수익 산점도
        fig_scatter = visualizer.plot_risk_return_scatter(analyses)
        fig_scatter.show()
        
        # 첫 번째 ETF의 구성종목
        if analyses[0].components:
            fig_pie = visualizer.plot_component_pie(analyses[0].components)
            fig_pie.show()
        
        # 요약 테이블
        fig_table = visualizer.create_summary_table(analyses[0])
        fig_table.show()
    
    # 8. 결과 저장
    print("\n💾 결과 저장 중...")
    output_dir = Path("./etf_analysis_output")
    output_dir.mkdir(exist_ok=True)
    
    for analysis in analyses:
        # JSON 저장
        json_path = output_dir / f"etf_{analysis.basic_info.code}_{datetime.now().strftime('%Y%m%d')}.json"
        with open(json_path, "w", encoding="utf-8") as f:
            # dataclass를 dict로 변환할 때 datetime 처리
            data = analysis.to_dict()
            # datetime 객체 문자열 변환
            if data["basic_info"]["listing_date"]:
                data["basic_info"]["listing_date"] = data["basic_info"]["listing_date"].isoformat()
            json.dump(data, f, ensure_ascii=False, indent=2, default=str)
        print(f"   ✅ {json_path}")
    
    print("\n🎉 분석 완료!")
    
    return analyses


# 실행
if __name__ == "__main__":
    results = main()



🔧 설정 로드 중...
🔐 토큰 검증 중...
✅ 토큰 유효

📊 ETF 분석 시작: 069500
1️⃣ 기본 정보 조회 중...
2️⃣ 호가 정보 조회 중...
3️⃣ 구성종목 조회 중...
   → 100개 종목 조회됨
4️⃣ 가격 히스토리 조회 중...
5️⃣ 위험 지표 계산 중...

✅ 분석 완료: 삼성자산운용(ETF) (069500)

📊 삼성자산운용(ETF) 요약
   현재가: 56,080원
   NAV: 56,134원 (괴리율: -0.096%)
   AUM: 10.84조원
   스프레드: 0.009%
   20일 평균 거래대금: 10,111억원
   1년 수익률: +79.2%
   연간 변동성: 19.8%
   MDD: -21.6%
   샤프비율: 1.058
   품질점수: 95/100

📊 ETF 분석 시작: 252670
1️⃣ 기본 정보 조회 중...
2️⃣ 호가 정보 조회 중...
3️⃣ 구성종목 조회 중...
   → 1개 종목 조회됨
4️⃣ 가격 히스토리 조회 중...
5️⃣ 위험 지표 계산 중...

✅ 분석 완료: 삼성자산운용(ETF) (252670)

📊 삼성자산운용(ETF) 요약
   현재가: 734원
   NAV: 734원 (괴리율: -0.025%)
   AUM: 1.57조원
   스프레드: 0.136%
   20일 평균 거래대금: 8,876억원
   1년 수익률: -71.8%
   연간 변동성: 40.0%
   MDD: -81.2%
   샤프비율: -1.129
   품질점수: 72/100

📈 시각화 생성 중...



💾 결과 저장 중...
   ✅ etf_analysis_output/etf_069500_20251126.json
   ✅ etf_analysis_output/etf_252670_20251126.json

🎉 분석 완료!


## 🔟 백테스팅 예시


In [10]:
def run_backtest_example():
    """백테스팅 예시"""
    
    config = KISConfig.from_env()
    client = KISAPIClient(config)
    analyzer = ETFAnalyzer(client)
    backtester = ETFBacktester(analyzer)
    
    # 데이터 준비
    etf_codes = ["069500", "229200", "305720"]
    prices = backtester.prepare_bt_data(etf_codes)
    
    if prices.empty:
        print("❌ 데이터 없음")
        return
    
    print(f"✅ 데이터 준비 완료: {len(prices)}일, {len(prices.columns)}개 종목")
    
    # 동일가중 백테스트
    result = backtester.run_simple_backtest(prices)
    
    if result:
        print("\n📊 동일가중 포트폴리오 결과:")
        print(f"   총 수익률: {result['total_return']:.1f}%")
        print(f"   CAGR: {result['cagr']:.1f}%")
        print(f"   MDD: {result['max_drawdown']:.1f}%")
        print(f"   샤프비율: {result['sharpe']:.3f}")
        print(f"   연간 변동성: {result['volatility']:.1f}%")
        
        # 누적 수익률 차트
        if "result_object" in result:
            result["result_object"].plot()

# run_backtest_example()  # 필요 시 주석 해제



## 📋 사용법 요약

```python
# 1. 환경 설정 (.env 파일 필요)
# KIS_URL_BASE=https://openapi.koreainvestment.com:9443
# KIS_APP_KEY=your_app_key
# KIS_APP_SECRET=your_app_secret  
# KIS_ACCESS_TOKEN=your_access_token

# 2. 클라이언트 초기화
config = KISConfig.from_env()
client = KISAPIClient(config)
analyzer = ETFAnalyzer(client)

# 3. ETF 분석
analysis = analyzer.analyze("069500")

# 4. 시각화
visualizer = ETFVisualizer()
fig = visualizer.plot_returns_comparison([analysis])
fig.show()

# 5. 백테스팅
backtester = ETFBacktester(analyzer)
prices = backtester.prepare_bt_data(["069500", "229200"])
result = backtester.run_simple_backtest(prices)
```
